In [1]:
from query_api import query_api
from schedule_parser import TimetableProcessor


In [2]:
processor = TimetableProcessor(database_type="sqlite")  # or "csv"
current_events = processor.format_events_as_llm_output(include_header=False)
print(current_events)

In [7]:
import time

processor = TimetableProcessor(database_type="sqlite")  # or "csv"
current_events = processor.format_events_as_llm_output(include_header=False)
user_prompt = input("你有什么想放进计划表的东西吗？")
print(f"你的输入：{user_prompt}")
start_time = time.time()
#response = query_api(user_prompt, current_events, model="deepseek-chat")
response = query_api(user_prompt, current_events, model="gpt-4o")
end_time = time.time()
print(f"模型推理时间：{end_time - start_time}秒")
print("\n模型回复：")
print(response)

你的输入：我明天下午三点半要去小组讨论，大概两小时
模型推理时间：10.023371458053589秒

模型回复：
新任务：小组讨论
小组讨论 → 拆分为：
- 准备小组讨论（预计需一小时，时段：2025-03-07 10:30-11:30）
- 参加小组讨论（预计需两小时，时段：2025-03-07 15:30-17:30）

日程建议：
事项: 写代码
日期: 2025-03-06
时间段: 19:30-20:30
类型: 任务事项
截止日期：2025-03-06
重要程度：3
变动：无

事项: 与周姐开会
日期: 2025-03-07
时间段: 09:00-10:00
类型: 固定日程
截止日期：2025-03-07
重要程度：5
变动：无

事项: 准备小组讨论
日期: 2025-03-07
时间段: 10:30-11:30
类型: 任务事项
截止日期：2025-03-07
重要程度：4
变动：新增

事项: 参加小组讨论
日期: 2025-03-07
时间段: 15:30-17:30
类型: 固定日程
截止日期：2025-03-07
重要程度：5
变动：新增


In [8]:
# 获取修改前的所有事件
old_events = processor.get_all_events()

# 进行一些修改操作...
summary = processor.process_events(response)
print(summary)

# 获取修改后的所有事件
new_events = processor.get_all_events()

# 显示变更
changes = processor.format_events_with_changes(old_events, new_events, include_header=True)
print(changes)

Extracted 4 events from LLM output
Event 1: 写代码 - 2025-03-06 - 无
Event 2: 与周姐开会 - 2025-03-07 - 无
Event 3: 准备小组讨论 - 2025-03-07 - 新增
Event 4: 参加小组讨论 - 2025-03-07 - 新增
{'added': 2, 'modified': 0, 'deleted': 0, 'unchanged': 2, 'skipped': 0, 'errors': [], 'warnings': []}
日程变更明细：

----------------------------------------

[ ] 事项: 写代码
    日期: 2025-03-06
    时间段: 19:30-20:30
    类型: 任务事项
    截止日期：2025-03-06
    重要程度：3

[ ] 事项: 与周姐开会
    日期: 2025-03-07
    时间段: 09:00-10:00
    类型: 固定日程
    截止日期：2025-03-07
    重要程度：5

[+] 事项: 准备小组讨论 (新增)
    日期: 2025-03-07
    时间段: 10:30-11:30
    类型: 任务事项
    截止日期：2025-03-07
    重要程度：4

[+] 事项: 参加小组讨论 (新增)
    日期: 2025-03-07
    时间段: 15:30-17:30
    类型: 固定日程
    截止日期：2025-03-07
    重要程度：5


In [6]:
# This gets all the events in the database
formatted_output = processor.format_events_as_llm_output()
print(formatted_output)

# This gets all the events for a specific date
today_events = processor.get_events_for_date("2025-03-03")
today_formatted = processor.format_events_as_llm_output(today_events)
#print(today_formatted)

In [5]:
# 使用当前时间删除过期事件
result = processor.delete_past_events()

# 查看删除结果
print(f"删除了 {result['deleted_count']} 个事件")
for event in result['deleted_events']:
    print(f"- {event['title']} ({event['date']} {event['time_range']})")

删除了 9 个事件
- 准备会议材料 (2025-03-04 08:00-10:00)
- 选择花束 (2025-03-05 18:00-18:30)
- 下单购买 (2025-03-05 18:30-19:00)
- 分析需求 (2025-03-04 10:30-11:30)
- 编写代码 (2025-03-04 11:00-13:00)
- 测试与调试 (2025-03-04 13:15-14:15)
- 吃早饭 (2025-03-04 07:00-07:30)
- 遛狗 (2025-03-04 07:30-08:00)
- 运动 (2025-03-04 20:30-21:30)
